# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), July 16, 2019**

In [1]:
using DataFrames
using BenchmarkTools

## Performance tips

### Access by column number is faster than by name

In [2]:
x = DataFrame(rand(5, 1000))
@btime $x[!, 500];
@btime $x.x500;

  4.199 ns (0 allocations: 0 bytes)
  14.328 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [3]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  83.402 ms (5999023 allocations: 122.06 MiB)


In [4]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  12.379 ms (45 allocations: 30.52 MiB)
  12.863 ms (45 allocations: 30.52 MiB)


In [5]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  12.582 ms (45 allocations: 30.52 MiB)


### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between `DataFrame` and `DataFrame!` (copying vs non-copying data frame creation)

In [6]:
function f1()
    x = DataFrame!([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame!(x) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  20.849 ms (1949837 allocations: 37.42 MiB)
  21.840 ms (1950037 allocations: 45.06 MiB)
  1.568 ms (937 allocations: 7.69 MiB)
  2.538 ms (1137 allocations: 15.32 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [7]:
x = DataFrame(rand(10^6, 5))
y = DataFrame(transpose(1.0:5.0))
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5)) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  13.669 ms (174 allocations: 38.16 MiB)
  166.582 ns (1 allocation: 16 bytes)
  193.721 ns (5 allocations: 80 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [8]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.625 ms (7 allocations: 7.63 MiB)
 categorical:
  18.108 ms (4 allocations: 608 bytes)
String
 raw:
  23.858 ms (4 allocations: 608 bytes)
 categorical:
  30.332 ms (4 allocations: 608 bytes)
Union{Missing, Int64}
 raw:
  10.390 ms (4 allocations: 624 bytes)
 categorical:
  18.695 ms (4 allocations: 608 bytes)
Union{Missing, String}
 raw:
  24.059 ms (4 allocations: 608 bytes)
 categorical:
  37.816 ms (4 allocations: 608 bytes)


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [9]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [10]:
@btime by($df, :x, v -> sum(v.y)) # traditional syntax, slow

  313.270 ms (196 allocations: 433.19 MiB)


,x,x1
,Char,Int64
1,'b',2504347
2,'d',2498415
3,'c',2499953
4,'a',2497285


In [11]:
@btime by($df, :x, :y=>sum) # use column selector

  218.989 ms (157 allocations: 356.89 MiB)


,x,y_sum
,Char,Int64
1,'b',2504347
2,'d',2498415
3,'c',2499953
4,'a',2497285


In [12]:
categorical!(df, :x);

In [13]:
@btime by($df, :x, :y=>sum)

  97.493 ms (176 allocations: 152.60 MiB)


,x,y_sum
,Categorical…,Int64
1,'a',2497285
2,'b',2504347
3,'c',2499953
4,'d',2498415


In [14]:
using PooledArrays

In [15]:
df.x = PooledArray{Char}(df.x)

10000000-element PooledArray{Char,UInt8,1,Array{UInt8,1}}:
 'b'
 'd'
 'b'
 'd'
 'c'
 'b'
 'b'
 'b'
 'c'
 'c'
 ⋮  
 'a'
 'c'
 'b'
 'b'
 'd'
 'c'
 'b'
 'a'
 'a'

In [16]:
@btime by($df, :x, :y=>sum)

  97.074 ms (166 allocations: 152.60 MiB)


,x,y_sum
,Char,Int64
1,'b',2504347
2,'d',2498415
3,'c',2499953
4,'a',2497285


### Use views instead of materializing a new DataFrame

In [17]:
x = DataFrame(rand(100, 1000))

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725,0.192599,0.00604097,0.581288,0.746245,0.385509,0.355455,0.929221,0.729291,0.778783,0.598102,0.498373,0.0278266,0.0462616,0.0804585,0.486005,0.829665,0.605489,0.922619,0.615983,0.975146,0.218467,0.214458,0.201525,0.388645,0.558404,0.163782,0.437683,0.897702,0.0696074,0.656873,0.096646,0.604482,0.826148,0.90222,0.213158,0.145633,0.715524,0.821202,0.851274,0.809465,0.938894,0.833493,0.0128205,0.911447,0.572925,0.492402,0.70685,0.391735,0.379956,0.894197,0.319608,0.282724,0.662529,0.751371,0.0515758,0.0554555,0.407253,0.714758,0.930454,0.34133,0.409226,0.0593862,0.943453,0.396289,0.611359,0.630375,0.0122984,0.948587,0.395367,0.912241,0.192444
2,0.815357,0.7703,0.239176,0.275748,0.183924,0.507341,0.284418,0.28708,0.942902,0.0887963,0.734511,0.379075,0.434635,0.474651,0.280465,0.180545,0.997093,0.58534,0.404039,0.0574775,0.829238,0.551627,0.0609086,0.866673,0.00203838,0.23122,0.100199,0.542427,0.6176,0.210846,0.169462,0.317331,0.120702,0.522714,0.771854,0.861075,0.607028,0.577105,0.0263973,0.508519,0.207115,0.260203,0.966495,0.950563,0.23515,0.807233,0.667959,0.469412,0.68944,0.303178,0.916198,0.175827,0.630586,0.617525,0.436164,0.30317,0.0596742,0.0982103,0.094269,0.774918,0.74536,0.45168,0.0928307,0.314255,0.837564,0.507878,0.633839,0.298748,0.501813,0.667421,0.244102,0.99187,0.320146,0.327764,0.0972044,0.865251,0.370263,0.207617,0.490622,0.128018,0.973084,0.769352,0.780947,0.258148,0.812933,0.83484,0.432996,0.511692,0.611335,0.877168,0.742428
3,0.816104,0.0311551,0.5683,0.704495,0.549385,0.471999,0.543525,0.878611,0.0553813,0.433413,0.721113,0.90562,0.822592,0.101155,0.215981,0.0727627,0.579711,0.579151,0.755325,0.632599,0.676199,0.921829,0.749605,0.981136,0.10249,0.953556,0.50945,0.646269,0.396988,0.932969,0.506011,0.387092,0.513568,0.548799,0.117301,0.749199,0.161664,0.507067,0.982289,0.0554118,0.0723973,0.296174,0.2617,0.0464874,0.0168145,0.938944,0.731108,0.464276,0.0282041,0.112549,0.745749,0.432314,0.693835,0.956663,0.177674,0.874948,0.587931,0.834267,0.751571,0.871607,0.118688,0.895995,0.688151,0.931742,0.0470647,0.356565,0.648014,0.314232,0.90112,0.837802,0.419339,0.126734,0.00268467,0.843003,0.505144,0.5602,0.0985052,0.861954,0.860644,0.204682,0.634035,0.553771,0.220832,0.416234,0.900075,0.095238,0.323409,0.461687,0.099201,0.043433,0.690487
4,0.0294096,0.602984,0.357492,0.588966,0.100176,0.163219,0.3719,0.0634015,0.57308,0.795895,0.89425,0.0849626,0.763744,0.239118,0.722183,0.856142,0.180864,0.581203,0.3304,0.520827,0.0140902,0.402648,0.492288,0.0419267,0.565855,0.185707,0.912092,0.276022,0.687733,0.152818,0.596199,0.246881,0.624406,0.86924,0.60933,0.662877,0.97319,0.112181,0.417401,0.384564,0.824204,0.936204,0.710395,0.962075,0.429801,0.781552,0.110724,0.238736,0.239962,0.8055

In [18]:
@btime $x[1:1, :]

  135.301 μs (1511 allocations: 194.41 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725,0.192599,0.00604097,0.581288,0.746245,0.385509,0.355455,0.929221,0.729291,0.778783,0.598102,0.498373,0.0278266,0.0462616,0.0804585,0.486005,0.829665,0.605489,0.922619,0.615983,0.975146,0.218467,0.214458,0.201525,0.388645,0.558404,0.163782,0.437683,0.897702,0.0696074,0.656873,0.096646,0.604482,0.826148,0.90222,0.213158,0.145633,0.715524,0.821202,0.851274,0.809465,0.938894,0.833493,0.0128205,0.911447,0.572925,0.492402,0.70685,0.391735,0.379956,0.894197,0.319608,0.282724,0.662529,0.751371,0.0515758,0.0554555,0.407253,0.714758,0.930454,0.34133,0.409226,0.0593862,0.943453,0.396289,0.611359,0.630375,0.0122984,0.948587,0.395367,0.912241,0.192444,0.69041,0.498202,0.157421,0.0875531,0.907202,0.671311,0.84797,0.990317,0.722969


In [19]:
@btime $x[1, :]

  24.698 ns (1 allocation: 32 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725,0.192599,0.00604097,0.581288,0.746245,0.385509,0.355455,0.929221,0.729291,0.778783,0.598102,0.498373,0.0278266,0.0462616,0.0804585,0.486005,0.829665,0.605489,0.922619,0.615983,0.975146,0.218467,0.214458,0.201525,0.388645,0.558404,0.163782,0.437683,0.897702,0.0696074,0.656873,0.096646,0.604482,0.826148,0.90222,0.213158,0.145633,0.715524,0.821202,0.851274,0.809465,0.938894,0.833493,0.0128205,0.911447,0.572925,0.492402,0.70685,0.391735,0.379956,0.894197,0.319608,0.282724,0.662529,0.751371,0.0515758,0.0554555,0.407253,0.714758,0.930454,0.34133,0.409226,0.0593862,0.943453,0.396289,0.611359,0.630375,0.0122984,0.948587,0.395367,0.912241,0.192444,0.69041,0.498202,0.157421,0.0875531,0.907202,0.671311,0.84797,0.990317,0.722969


In [20]:
@btime view($x, 1:1, :)

  25.026 ns (1 allocation: 48 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725,0.192599,0.00604097,0.581288,0.746245,0.385509,0.355455,0.929221,0.729291,0.778783,0.598102,0.498373,0.0278266,0.0462616,0.0804585,0.486005,0.829665,0.605489,0.922619,0.615983,0.975146,0.218467,0.214458,0.201525,0.388645,0.558404,0.163782,0.437683,0.897702,0.0696074,0.656873,0.096646,0.604482,0.826148,0.90222,0.213158,0.145633,0.715524,0.821202,0.851274,0.809465,0.938894,0.833493,0.0128205,0.911447,0.572925,0.492402,0.70685,0.391735,0.379956,0.894197,0.319608,0.282724,0.662529,0.751371,0.0515758,0.0554555,0.407253,0.714758,0.930454,0.34133,0.409226,0.0593862,0.943453,0.396289,0.611359,0.630375,0.0122984,0.948587,0.395367,0.912241,0.192444,0.69041,0.498202,0.157421,0.0875531,0.907202,0.671311,0.84797,0.990317,0.722969


In [21]:
@btime $x[1:1, 1:20]

  4.486 μs (55 allocations: 7.23 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725


In [22]:
@btime $x[1, 1:20]

  27.766 ns (2 allocations: 80 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725


In [23]:
@btime view($x, 1:1, 1:20)

  30.614 ns (2 allocations: 96 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.691238,0.502941,0.113297,0.328738,0.13378,0.441278,0.685002,0.0538219,0.0761798,0.769235,0.284291,0.171352,0.754563,0.648736,0.0953225,0.775889,0.841043,0.311933,0.56814,0.577725
